<a href="https://www.kaggle.com/code/dewilliams/multi-agent-life-advisory-board?scriptVersionId=232946120" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Life Advisory Board - Multi-Agent Interactive Chat System

## Use Case
This system is designed for individuals seeking personalized, multi-faceted advice on life decisions—ranging from fitness and financial planning to career transitions and personal growth. It leverages a team of AI advisors (Financial, Career, Health, Personal) to provide holistic guidance.

## Problem
People often face complex decisions requiring diverse perspectives (e.g., health, finance, personal fulfillment), but single-source advice lacks breadth. Existing systems may not tailor responses to user thinking styles (analytical, practical, relational, experimental) or evaluate response quality systematically.

## Solution
The Life Advisory Board uses a multi-agent architecture with domain-specific knowledge bases (from PDFs). Advisors discuss queries collaboratively, tailoring advice to the user’s inferred style. A supervisor aggregates inputs into a unified response. Quality is assessed post-session using Google’s Text Quality rubric (coherence, fluency, instruction following, groundedness, verbosity), scoring 1-5, with non-participating advisors marked "N/A."

## How to Use
1. **Setup**: Run in Kaggle with the `advisor-notes` dataset (Financial_Advice.pdf, Career_Advice.pdf, Health_Advice.pdf, Personal_Advice.pdf) added via "Add Input."
2. **Interact**: Execute the notebook, enter queries (e.g., "Should I start a business?"), and receive advice with background discussion.
3. **Exit & Evaluate**: Type "exit" to end the chat and view quality evaluations for all responses.
4. **Interpret**: Advice is ~150 words, discussion ~100 words per turn. Evaluations at end detail scores (5-1) for coherence, fluency, etc.

## Setup and Imports

In [1]:
# Install required packages
!pip install -qU langchain-google-genai==2.0.0 langgraph==0.3.0 PyPDF2

In [2]:
# Run it a 2nd time due to issues with pip in Kaggle.
!pip install -qU langchain-google-genai==2.0.0 langgraph==0.3.0 PyPDF2

## Implementation

In [20]:
# Import necessary libraries
import os
import json
from typing import Dict, Annotated, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import PyPDF2
from kaggle_secrets import UserSecretsClient
from langgraph.graph.message import add_messages

# Configuration
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.2, max_tokens=1000)
INPUT_DIR = "/kaggle/input/advisor-notes/"

# Global chat history to store interactions for evaluation at exit
chat_history = []

# Core Function: Loads knowledge base from PDF files into domain-specific corpora
# Purpose: Extracts text from PDFs in INPUT_DIR, organizing it into a dictionary by domain (financial, career, health, personal) for advisor use
# Usage: Called at startup to initialize corpora; each advisor accesses its domain-specific content
def load_knowledge_base(directory: str) -> Dict[str, str]:
    print(f"Loading knowledge base from {directory}...")
    corpora = {"financial": "", "career": "", "health": "", "personal": ""}
    file_mapping = {
        "Financial_Advice.pdf": "financial",
        "Career_Advice.pdf": "career",
        "Health_Advice.pdf": "health",
        "Personal_Advice.pdf": "personal",
        "original_corpus.pdf": "career"
    }
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            filepath = os.path.join(directory, filename)
            domain = file_mapping.get(filename, "personal")
            try:
                with open(filepath, "rb") as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page in pdf_reader.pages:
                        text = page.extract_text() or ""
                        corpora[domain] += f"\n--- {filename} ---\n{text}"
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    for domain, content in corpora.items():
        print(f"{domain} corpus (first 50 chars): {content[:50]}...")
    return corpora

# Core Function: Infers user's thinking style from input
# Purpose: Analyzes query keywords to determine user focus (analytical, practical, relational, experimental) for tailored advice
# Usage: Called per query in run_interactive_chat to set state["style"], guiding advisor responses
def infer_style(input_text: str) -> str:
    scores = {"analytical": 0, "practical": 0, "relational": 0, "experimental": 0}
    text = input_text.lower()
    if any(word in text for word in ["data", "profit", "numbers", "calculate", "roi", "cost", "value", "stats"]):
        scores["analytical"] += 1
    if any(word in text for word in ["how", "steps", "plan", "process", "do", "start", "manage", "way"]):
        scores["practical"] += 1
    if any(word in text for word in ["people", "feel", "team", "family", "support", "like", "happy", "stress", "well"]):
        scores["relational"] += 1
    if any(word in text for word in ["future", "potential", "ideas", "vision", "grow", "imagine", "could", "might"]):
        scores["experimental"] += 1
    return max(scores, key=scores.get) if max(scores.values()) > 0 else "experimental"

# Tool Function: Evaluates the quality of an LLM response
# Purpose: Assesses advisor and supervisor responses using Google’s Text Quality rubric (coherence, fluency, instruction following, groundedness, verbosity)
# Usage: Called at chat exit in run_interactive_chat to score each participating advisor’s input and the final advice (5-1 scale)
def evaluate_response(prompt: str, response: str) -> str:
    eval_prompt = f"""
    You are an expert evaluator assessing Text Quality (clarity, accuracy, engagement) based on:
    - Coherence: Logical, organized, easy to follow.
    - Fluency: Smooth, grammatical, appropriate vocabulary.
    - Instruction Following: Meets prompt requirements.
    - Groundedness: Stays within provided context.
    - Verbosity: Concise yet sufficient, not overly wordy/brief.
    
    Score from 5 (very good) to 1 (very bad):
    5: Exceptional in all criteria.
    4: Good, minor flaws.
    3: Adequate, some issues.
    2: Poor, significant flaws.
    1: Very poor, fails most criteria.

    User Prompt: "{prompt}"
    Response: "{response}"

    Step 1: Assess each criterion.
    Step 2: Assign a score with step-by-step rationale.
    """
    print(f"Evaluating prompt: {eval_prompt[:100]}...")  # Debug print
    eval_response = llm.invoke([SystemMessage(content=eval_prompt), HumanMessage(content="Evaluate this response.")])
    return eval_response.content.strip()

# Define agent state class
class AgentState(Dict):
    messages: Annotated[List, add_messages]  # Stores user query and final advice
    next: str  # Next node in the graph
    style: str  # User’s inferred thinking style
    discussion: List[str]  # List of advisor contributions per query

# Core Function: Financial Advisor node
# Purpose: Provides financial perspective on the user’s query using the financial corpus
# Usage: Triggered by supervisor based on keywords (e.g., "money"), adds input to discussion, passes to Career Advisor
def financial_advisor(state: AgentState, corpora: Dict[str, str]) -> AgentState:
    user_input = state["messages"][0].content
    style = state["style"]
    system_message = SystemMessage(content=f"""
    You are a Financial Advisor with access to: {corpora['financial'][:2000]}.
    Provide input on: "{user_input}" tailored to the user's focus without mentioning thinking styles:
    - Facts: Lead with data/metrics (e.g., costs, returns) and include examples.
    - Process: Start with steps (e.g., what to do next) with specific actions.
    - People: Emphasize relationships/emotions (e.g., impact on others) with examples.
    - Ideas: Highlight vision/possibilities (e.g., future potential) with concrete scenarios.
    Keep it concise (under 100 words). Base it on your financial expertise, citing examples from the corpus.
    """)
    messages = [system_message, HumanMessage(content=user_input)]
    response = llm.invoke(messages)
    discussion = state["discussion"] + [f"Financial Advisor: {response.content}"]
    return {"discussion": discussion, "next": "Career_Advisor"}

# Core Function: Career Advisor node
# Purpose: Offers career-related advice on the user’s query using the career corpus
# Usage: Follows Financial Advisor, builds on prior input if relevant, passes to Health Advisor
def career_advisor(state: AgentState, corpora: Dict[str, str]) -> AgentState:
    user_input = state["messages"][0].content
    style = state["style"]
    prev_discussion = "\n".join(state["discussion"])
    system_message = SystemMessage(content=f"""
    You are a Career Advisor with access to: {corpora['career'][:2000]}.
    Previous input: {prev_discussion}.
    Provide input on: "{user_input}" tailored to the user's focus without mentioning thinking styles:
    - Facts: Lead with data/metrics (e.g., salary, stats) and include examples.
    - Process: Start with steps (e.g., career planning) with specific actions.
    - People: Emphasize relationships/emotions (e.g., work-life balance) with examples.
    - Ideas: Highlight vision/possibilities (e.g., career growth) with concrete scenarios.
    Keep it concise (under 100 words). Build on prior input if relevant, citing examples from the corpus.
    """)
    messages = [system_message, HumanMessage(content=user_input)]
    response = llm.invoke(messages)
    discussion = state["discussion"] + [f"Career Advisor: {response.content}"]
    return {"discussion": discussion, "next": "Health_Advisor"}

# Core Function: Health Advisor node
# Purpose: Provides health-focused advice on the user’s query using the health corpus
# Usage: Follows Career Advisor, builds on prior input if relevant, passes to Personal Advisor
def health_advisor(state: AgentState, corpora: Dict[str, str]) -> AgentState:
    user_input = state["messages"][0].content
    style = state["style"]
    prev_discussion = "\n".join(state["discussion"])
    system_message = SystemMessage(content=f"""
    You are a Health Advisor with access to: {corpora['health'][:2000]}.
    Previous input: {prev_discussion}.
    Provide input on: "{user_input}" tailored to the user's focus without mentioning thinking styles:
    - Facts: Lead with data/metrics (e.g., stats, benefits) and include examples.
    - Process: Start with steps (e.g., health routines) with specific actions.
    - People: Emphasize relationships/emotions (e.g., family impact) with examples.
    - Ideas: Highlight vision/possibilities (e.g., wellness goals) with concrete scenarios.
    Keep it concise (under 100 words). Build on prior input if relevant, citing examples from the corpus.
    """)
    messages = [system_message, HumanMessage(content=user_input)]
    response = llm.invoke(messages)
    discussion = state["discussion"] + [f"Health Advisor: {response.content}"]
    return {"discussion": discussion, "next": "Personal_Advisor"}

# Core Function: Personal Advisor node
# Purpose: Offers personal growth and well-being advice on the user’s query using the personal corpus
# Usage: Follows Health Advisor, builds on prior input if relevant, passes to Supervisor
def personal_advisor(state: AgentState, corpora: Dict[str, str]) -> AgentState:
    user_input = state["messages"][0].content
    style = state["style"]
    prev_discussion = "\n".join(state["discussion"])
    system_message = SystemMessage(content=f"""
    You are a Personal Advisor with access to: {corpora['personal'][:2000]}.
    Previous input: {prev_discussion}.
    Provide input on: "{user_input}" tailored to the user's focus without mentioning thinking styles:
    - Facts: Lead with data/metrics (e.g., time, resources) and include examples.
    - Process: Start with steps (e.g., personal plans) with specific actions.
    - People: Emphasize relationships/emotions (e.g., connections) with examples.
    - Ideas: Highlight vision/possibilities (e.g., life goals) with concrete scenarios.
    Keep it concise (under 100 words). Build on prior input if relevant, citing examples from the corpus.
    """)
    messages = [system_message, HumanMessage(content=user_input)]
    response = llm.invoke(messages)
    discussion = state["discussion"] + [f"Personal Advisor: {response.content}"]
    return {"discussion": discussion, "next": "Supervisor"}

# Core Function: Supervisor node
# Purpose: Routes queries to advisors and aggregates their inputs into a final response
# Usage: Entry point for each query; on first pass, routes based on keywords; on return, synthesizes discussion into advice
def supervisor(state: AgentState) -> AgentState:
    last_message = state["messages"][-1].content.lower()
    if len(state["messages"]) == 1 and state.get("discussion", []) == []:  # Initial query
        if any(keyword in last_message for keyword in ["money", "finance", "invest", "business"]):
            return {"next": "Financial_Advisor"}
        elif any(keyword in last_message for keyword in ["job", "career", "work"]):
            return {"next": "Career_Advisor"}
        elif any(keyword in last_message for keyword in ["health", "wellness", "fit", "fitness", "strength", "workout", "weight", "fat", "muscle"]):
            return {"next": "Health_Advisor"}
        else:
            return {"next": "Personal_Advisor"}
    else:  # Aggregate discussion
        discussion = "\n".join(state["discussion"])
        system_message = SystemMessage(content=f"""
        You are the Supervisor of the Life Advisory Board. Advisors provided:
        {discussion}
        Aggregate their inputs into a concise, unified response (under 200 words) for: "{last_message}".
        Ensure it’s clear, balanced, and actionable, reflecting all perspectives.
        """)
        messages = [system_message, HumanMessage(content=last_message)]
        response = llm.invoke(messages)
        chat_history.append({
            "prompt": last_message,
            "discussion": state["discussion"],
            "response": response.content
        })
        full_response = f"{response.content}\n\nBackground Discussion:\n{discussion}"
        return {"messages": [AIMessage(content=full_response)], "next": "END", "discussion": []}

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("Supervisor", supervisor)
workflow.add_node("Financial_Advisor", lambda state: financial_advisor(state, corpora))
workflow.add_node("Career_Advisor", lambda state: career_advisor(state, corpora))
workflow.add_node("Health_Advisor", lambda state: health_advisor(state, corpora))
workflow.add_node("Personal_Advisor", lambda state: personal_advisor(state, corpora))

workflow.set_entry_point("Supervisor")
workflow.add_conditional_edges(
    "Supervisor",
    lambda state: state["next"],
    {
        "Financial_Advisor": "Financial_Advisor",
        "Career_Advisor": "Career_Advisor",
        "Health_Advisor": "Health_Advisor",
        "Personal_Advisor": "Personal_Advisor",
        "END": END
    }
)
workflow.add_edge("Financial_Advisor", "Career_Advisor")
workflow.add_edge("Career_Advisor", "Health_Advisor")
workflow.add_edge("Health_Advisor", "Personal_Advisor")
workflow.add_edge("Personal_Advisor", "Supervisor")

app = workflow.compile()

# Load knowledge base
corpora = load_knowledge_base(INPUT_DIR)

# Core Function: Runs interactive chat with background evaluation
# Purpose: Manages user interaction, collects chat history, and evaluates quality at exit
# Usage: Main entry point; runs until "exit," then outputs all advice evaluations
def run_interactive_chat():
    print("Welcome to the Life Advisory Board! Ask anything (type 'exit' to quit):")
    all_advisors = ["Financial Advisor", "Career Advisor", "Health Advisor", "Personal Advisor"]
    while True:
        query = input("> ")
        if query.lower() == "exit":
            print("Goodbye!")
            print("\nFinal Evaluation of All Responses:")
            for entry in chat_history:
                print(f"\nPrompt: {entry['prompt']}")
                participating_advisors = [contrib.split(": ", 1)[0] for contrib in entry["discussion"]]
                for advisor in all_advisors:
                    if advisor in participating_advisors:
                        advice = next(contrib.split(": ", 1)[1] for contrib in entry["discussion"] if contrib.startswith(advisor))
                        eval_result = evaluate_response(entry["prompt"], advice)
                        print(f"{advisor}: {eval_result}")
                    else:
                        print(f"{advisor}: N/A - Did not participate in this session")
                print(f"Supervisor: {evaluate_response(entry['prompt'], entry['response'])}")
            break
        initial_state = {
            "messages": [HumanMessage(content=query)],
            "next": "Supervisor",
            "style": infer_style(query),
            "discussion": []
        }
        result = app.invoke(initial_state)
        print(f"Advice: {result['messages'][-1].content}\n")

# Start the chat
run_interactive_chat()

Loading knowledge base from /kaggle/input/advisor-notes/...
financial corpus (first 50 chars): 
--- Financial_Advice.pdf ---
Financial Advice Cor...
career corpus (first 50 chars): 
--- Career_Advice.pdf ---
Career Advice Corpus 1....
health corpus (first 50 chars): 
--- Health_Advice.pdf ---
Health Advice Corpus 1....
personal corpus (first 50 chars): 
--- Personal_Advice.pdf ---
Personal Advice Corpu...
Welcome to the Life Advisory Board! Ask anything (type 'exit' to quit):


>  I am tired and want to build more energy and strength


Advice: Feeling tired? Let's boost your energy and strength!  A combined approach from our Health and Personal Advisors recommends starting with 30 minutes of daily walking, gradually increasing intensity. Prioritize 7-8 hours of sleep nightly.  Incorporate strength training 2-3 times a week, even short 15-minute sessions are beneficial.  Focus on a balanced diet rich in protein and micronutrients.  Small, consistent changes are key.  Sufficient protein and nutrient-dense meals will fuel your body and improve your overall well-being. This plan will enhance your energy levels and build strength over time. Remember, consistency is crucial for long-term success.


Background Discussion:
Health Advisor: Feeling tired? Let's boost your energy and strength!  Prioritize sleep (Health Advice Corpus #2) and incorporate strength training (Corpus #4), crucial for longevity and metabolic health (Corpus #11, #13).  Start with daily 30-minute walks, gradually increasing intensity.  Improve your diet

>  Thanks.


Advice: Thank you.  We appreciate your engagement.  To move forward, consider three key perspectives:  Firstly, identify activities that bring you profound joy ("bliss," Campbell). Secondly, nurture and strengthen your most meaningful relationships ("true friendship," Aquinas).  Finally, define concrete, long-term goals that encompass your desired level of wealth and freedom (Ravikant).  Reflect on which of these – joy, relationships, or goals – currently requires the most attention.  Prioritizing one area doesn't preclude the others; a balanced approach integrating all three will likely yield the greatest fulfillment.  Consider journaling or brainstorming to clarify your priorities and create an actionable plan.


Background Discussion:
Personal Advisor: You're welcome!  To build on this, consider what resonates most:  Focusing on your "bliss" (Campbell #1) might unlock opportunities.  Alternatively, prioritizing strong relationships ("true friendship," Aquinas #14) can bring immense 

>  exit


Goodbye!

Final Evaluation of All Responses:

Prompt: i am tired and want to build more energy and strength
Financial Advisor: N/A - Did not participate in this session
Career Advisor: N/A - Did not participate in this session
Evaluating prompt: 
    You are an expert evaluator assessing Text Quality (clarity, accuracy, engagement) based on:
  ...
Health Advisor: **Step 1: Criterion Assessment**

* **Coherence:** The response is logically organized, progressing from acknowledging the user's tiredness to suggesting actionable steps for improving energy and strength.  The suggestions flow naturally.

* **Fluency:** The language is smooth, grammatically correct, and uses appropriate vocabulary. The tone is encouraging and supportive.

* **Instruction Following:** The response directly addresses the user's desire to build energy and strength. It provides concrete, relevant advice.

* **Groundedness:** The response uses numbered references ("Corpus #...") suggesting it draws upon external s